# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sns
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = "../output_data/cities.csv"
df = pd.read_csv(filepath)
df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,81.00,83,40,9.17,US,1604608907
1,Alyangula,-13.85,136.42,80.60,88,19,5.82,AU,1604609368
2,Nikolskoye,59.70,30.79,46.40,75,20,17.90,RU,1604609084
3,Romny,50.75,33.47,40.08,94,0,7.70,UA,1604609371
4,Hong Kong,22.29,114.16,72.00,81,0,1.01,HK,1604609372


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humids = df["Humidity"].astype(float)

In [5]:
coords = df[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=400)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,81.00,83,40,9.17,US,1604608907
1,Alyangula,-13.85,136.42,80.60,88,19,5.82,AU,1604609368
2,Nikolskoye,59.70,30.79,46.40,75,20,17.90,RU,1604609084
3,Romny,50.75,33.47,40.08,94,0,7.70,UA,1604609371
4,Hong Kong,22.29,114.16,72.00,81,0,1.01,HK,1604609372


In [7]:
#creating mask for ideal weather conditions
mask = (df['Max Temp'] < 85) & (df['Max Temp'] > 70) & (df["Wind Speed"] < 7) & (df["Cloudiness"] <30) & (df["Humidity"] < 50)
#creating new df on said conditions
df_sub = df.loc[mask].reset_index(drop=True)
df_sub.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Santa María Rayón,19.14,-99.58,75.20,15,5,4.70,MX,1604609451
1,Bonito,-21.12,-56.48,84.85,40,0,3.22,BR,1604609384
2,Indianola,41.36,-93.56,73.00,43,1,3.58,US,1604609576
3,Taoudenni,22.68,-3.98,81.70,16,19,5.91,ML,1604609647
4,Payson,40.04,-111.73,73.99,38,1,5.82,US,1604609710


In [8]:
df_sub.head(20)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Santa María Rayón,19.14,-99.58,75.20,15,5,4.70,MX,1604609451
1,Bonito,-21.12,-56.48,84.85,40,0,3.22,BR,1604609384
2,Indianola,41.36,-93.56,73.00,43,1,3.58,US,1604609576
3,Taoudenni,22.68,-3.98,81.70,16,19,5.91,ML,1604609647
4,Payson,40.04,-111.73,73.99,38,1,5.82,US,1604609710
5,Farmington,36.73,-108.22,73.00,21,1,5.82,US,1604609646
6,Samfya,-11.36,29.56,75.38,32,20,5.97,ZM,1604609794
7,Lafiagi,8.87,5.42,82.78,43,24,0.81,NG,1604609882
8,Laguna,38.42,-121.42,79.00,33,1,2.24,US,1604609935
9,Porterville,36.07,-119.02,79.00,35,1,4.70,US,1604609937


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=df_sub.City)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=400)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [10]:
hotel_df = df_sub

#inits
names = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
              
        name = results[0]['name']
              
        names.append(name)
              
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        
    print("------------")
    
hotel_df["Hotel Name"] = names

Retrieving Results for Index 0: Santa María Rayón.
Closest Hotel is Gym Vic Diaz.
------------
Retrieving Results for Index 1: Bonito.
Closest Hotel is Terra agropecuária.
------------
Retrieving Results for Index 2: Indianola.
Closest Hotel is Misty SOLDwisch.
------------
Retrieving Results for Index 3: Taoudenni.
Closest Hotel is Ksar de Smida.
------------
Retrieving Results for Index 4: Payson.
Closest Hotel is Memorial Park Pickleball Courts.
------------
Retrieving Results for Index 5: Farmington.
Closest Hotel is Airport Auto Center.
------------
Retrieving Results for Index 6: Samfya.
Closest Hotel is Twinstar Guest House.
------------
Retrieving Results for Index 7: Lafiagi.
Closest Hotel is Bindofu Lafiagi Health Post.
------------
Retrieving Results for Index 8: Laguna.
Closest Hotel is Home Maintenance of Elk Grove.
------------
Retrieving Results for Index 9: Porterville.
Closest Hotel is R&S Barbecue and Filipino Foods.
------------
Retrieving Results for Index 10: Ségou

In [11]:
hotel_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Santa María Rayón,19.14,-99.58,75.20,15,5,4.70,MX,1604609451,Gym Vic Diaz
1,Bonito,-21.12,-56.48,84.85,40,0,3.22,BR,1604609384,Terra agropecuária
2,Indianola,41.36,-93.56,73.00,43,1,3.58,US,1604609576,Misty SOLDwisch
3,Taoudenni,22.68,-3.98,81.70,16,19,5.91,ML,1604609647,Ksar de Smida
4,Payson,40.04,-111.73,73.99,38,1,5.82,US,1604609710,Memorial Park Pickleball Courts


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
# Display figure
fig = gmaps.figure()

figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(height='420px'))